# NN-based predictive model
(Based on Cen 2009)
This model is used to explain the power law in learning.  In this notebook we try to build a neuralised version of the AFM and train it using simulated data.  The aim of using the AFM is to disentangle the latent traits that make up the overall score going into the sigmoid probability estimator.

The model is compensatory, which is a weakness.

The NN takes the following values:
- estimate of user params
- num of attempts
- pass/fail indicator

and returns:
- estimate of question params


In [259]:
from collections import defaultdict, Counter
from copy import copy
from math import exp, sqrt, log
from random import random, shuffle, choice, randint, uniform
import numpy
import math

from keras import Input, Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.constraints import non_neg, max_norm
from numpy import array, mean, ones
from pandas import concat
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM, multiply, subtract, add, Activation, Lambda, Flatten
from keras.layers import Dense, concatenate, MaxPooling1D, LocallyConnected1D, Reshape, Dropout
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras import constraints

import tensorflow as tf

from utils import generate_student_name
import random

from matplotlib import pyplot as plt

n_traits = 3


In [260]:
generate_student_name()

'KYD PHYCH '

In [261]:
from keras import backend as K
from keras.constraints import Constraint
from keras.engine.topology import Layer
from keras import initializers

class WeightClip(Constraint):
    '''Clips the weights incident to each hidden unit to be inside a range
    '''
    def __init__(self, min_w=0, max_w=4):
        self.min_w = min_w
        self.max_w = max_w

    def __call__(self, p):
        return K.clip(p, self.min_w, self.max_w)

    def get_config(self):
        return {'name': self.__class__.__name__,
                'min_w': self.min_w,
                'max_w': self.max_w }


class ProductLayer(Layer):

    def __init__(self, output_dim, kernel_constraint=WeightClip(min_w=-4.0, max_w=4.0), minv=-4,maxv=4, **kwargs):
        
        self.output_dim = output_dim
        super(ProductLayer, self).__init__(**kwargs)
        self.kernel_constraint= constraints.get(kernel_constraint)
        self.min_v = minv
        self.max_v = maxv

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1, self.output_dim),
                                      initializer=initializers.RandomUniform(minval=self.min_v,maxval=self.max_v),
#                                       initializer=initializers.Constant(value=2.0),
                                      trainable=True,
                                      constraint=self.kernel_constraint)
        
        super(ProductLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        p = x * self.kernel
        print("shape p", p.shape)
        return p

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
    
class DifferenceLayer(Layer):

    def __init__(self, output_dim, kernel_constraint=WeightClip(min_w=-4.0, max_w=4.0), minv=-4,maxv=4, invert=False, **kwargs):
        
        self.output_dim = output_dim
        super(DifferenceLayer, self).__init__(**kwargs)
        self.kernel_constraint= constraints.get(kernel_constraint)
        self.min_v = minv
        self.max_v = maxv
        self.invert = invert

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        initialiser = initializers.RandomUniform(minval=self.min_v,maxval=self.max_v)
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1, self.output_dim),
                                      initializer=initialiser,
#                                       initializer=initializers.Constant(value=2.0),
                                      trainable=True,
                                      constraint=self.kernel_constraint)
        super(DifferenceLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        if self.invert:
            x = tf.Print(x, [x], message="x is:", first_n=-1, summarize=1024)
            k = tf.Print(self.kernel, [self.kernel], message="- kernel is:", first_n=-1, summarize=1024)
            p = x - k
        else:
            k = tf.Print(self.kernel, [self.kernel], message="kernel is:", first_n=-1, summarize=1024)
            x = tf.Print(x, [x], message="- x is:", first_n=-1, summarize=1024)
            p = k - x
#         p = K.print_tensor(p, message="p is:")
        p =  tf.Print(p, [p], message="p is:", first_n=-1, summarize=1024)
        print("shape p", p.shape)
        return p

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [262]:
for z in [-20, -10, -4,-3,-2,-1,0,1,2,3,4]:
    print(z, 1/(1+exp(-z)) )

# q_p_avg = 0.45
n_active_traits = n_traits
q_p_easiest = 0.9999
q_p_hardest = 0.0001
mid = (q_p_easiest + q_p_hardest)/2.0

# pr_k_avg = q_p_avg**(1/n_traits)
# print("pr k avg:", pr_k_avg)

pr_k_easiest = q_p_easiest**(1/n_active_traits)
pr_k_hardest = q_p_hardest**(1/n_active_traits)
pr_k_mid = mid**(1/n_traits)

inv_sigmoid = lambda pr : ( -log((1/pr) -1) )
easy_comp_del = inv_sigmoid(pr_k_easiest)
hard_comp_del = inv_sigmoid(pr_k_hardest)

offset = (easy_comp_del - hard_comp_del)/2
beta_min = 0
beta_max = round(offset,1)
theta_min = round(easy_comp_del - offset,1)
theta_max = round(easy_comp_del,1)

# beta_min = 0
# beta_max = 10
# theta_min = 5
# theta_max = 15


print(beta_min, beta_max)
print(theta_min, theta_max)

worst_comp_pr = 1/(1+exp(-(theta_min - beta_max)))
best_comp_pr = 1/(1+exp(-(theta_max - beta_min)))

print("worst cmp chance=", worst_comp_pr)
print("best cmp chance=", best_comp_pr)

print("worst Pr=", worst_comp_pr**n_active_traits)
print("best Pr=", best_comp_pr**n_active_traits)


nom = array([
    [1,2,3],
    [4,5,6],
    [7,8,9],
])

sel = nom[[0,2]]
print(sel)
print(sel.shape)


-20 2.0611536181902037e-09
-10 4.5397868702434395e-05
-4 0.01798620996209156
-3 0.04742587317756678
-2 0.11920292202211755
-1 0.2689414213699951
0 0.5
1 0.7310585786300049
2 0.8807970779778823
3 0.9525741268224334
4 0.9820137900379085
0 6.7
3.6 10.3
worst cmp chance= 0.043107254941086116
best cmp chance= 0.9999663680359613
worst Pr= 8.0103428359313e-05
best Pr= 0.9998991075011728
[[1 2 3]
 [7 8 9]]
(2, 3)


In [263]:
class BigTable(Layer):

    def __init__(self, _dim, min_w=0, max_w=10, **kwargs):
        self.dim = _dim
        self.limits = (min_w, max_w)
        kc =kernel_constraint=WeightClip(min_w, max_w)
        self.kernel_constraint= constraints.get(kc)
        super(BigTable, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        min_w, max_w = self.limits
        av_w = (min_w + max_w)/2.0
        initialiser = initializers.RandomUniform(min_w, max_w)
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(self.dim),
                                      initializer=initialiser,
                                      trainable=True)#,
#                                       constraint=self.kernel_constraint)
        print("kk", self.kernel.shape)
        super(BigTable, self).build(input_shape)  # Be sure to call this at the end

    def call(self, selector):
        print("selector shape", selector.shape)
        selector = K.flatten(selector)
        print("flat selector shape", selector.shape)
        print("call kk", self.kernel.shape)
        selector = tf.Print(selector, [selector], message="selector is:", first_n=-1, summarize=1024)
# #         row = tf.gather(self.kernel, [selector], axis=0)
#         rows = self.kernel[selector]
#         rows = tf.gather(self.kernel, selector, axis=1)
        rows = K.gather(self.kernel, selector)
#         rows = K.reshape(rows, (-1,self.dim[1]))
#         rows = K.flatten(rows)
# #         row = self.kernel
        rows = tf.Print(rows, [rows], message="row is:", first_n=-1, summarize=1024)
#         rows = self.kernel
        print("'rows' shape,",rows.shape)
        return rows

    def compute_output_shape(self, input_shape):
        return ((None, self.dim[1]))


In [264]:
class Question():
    def __init__(self, qix, min_diff, max_diff, nt=None, nnw=None, optimiser=None):
        #self.MAX_BETA = 15
        self.id = qix
#         no_dummies = randint(1,(nt-1))
        no_live = nt
#         print("no_dummies=",no_dummies)
        not_present= -10
#         min_diff = 0
#         max_diff = 10
        self.betas = [ not_present for _ in range(nt) ]
        choices = random.sample(range(nt), no_live)
        for c in choices:
            self.betas[c] = round(random.randint(10*min_diff,10*max_diff)/10,1)
#             self.betas = [ round(random.randint(10*min_diff,10*max_diff)/10,1) for _ in range(nt)]


In [265]:
class Student():
    def __init__(self, psix, min_abil, max_abil, nt=None, nnw=None, optimiser=None):
        #self.MAX_BETA = 15
        self.id = psix
        self.name = generate_student_name()
#         min_abil = 0
#         max_abil = 10
        self.thetas = [ round(random.randint(10*min_abil, 10*max_abil)/10,1) for _ in range(nt) ]
#         self.mastery = [0 for _ in range(nq)]
#         self.o_practice = [0 for _ in range(nq)]
#         self.h_practice = [0 for _ in range(nt)]
        #print("Made q with betas:", self.betas)


In [266]:
def attempt_q(student: Student, q: Question, verbose=False):
    p = calculate_pass_probability(student.thetas, q.betas, verbose=verbose)
    this_att = uniform(0,1)
    if (this_att <= p):
        passed=1.0
#         student.mastery[q.id] = 1
    else:
        passed=0.0

#     passed = 1 if p>=0.5 else 0

    return passed

In [267]:
def calculate_pass_probability(thetas, betas, verbose=False):
    # additive factors model is:
    # p_pass = 1 / 1 + exp(-z)
    # where z = a + sum[1:n]( -b + gT )
    
    p_pass = 1.0
#     print("th,b",thetas,betas)
    for th,b in zip(thetas,betas):
        z = (th-b)
        p_pass_step = 1.0 / (1.0 + exp(-z))
#         print(th,"vs",b,"->",z,": ", p_pass_step)
        p_pass *= p_pass_step # simple conjunctive model of success!
    try:
        if verbose:
            print("p_pass={}".format(p_pass))
    except OverflowError:
        p_pass = 0.0
    #print("real p_pass = {}".format(p_pass))
    return p_pass
    

In [268]:
qopt = Adam()

def create_qs(n_qs, nt=n_traits, nnw=n_traits, optimiser=qopt, seed=666):
    random.seed(seed)
    master_qs = [Question(qix, beta_min,beta_max, nt=nt, nnw=nnw, optimiser=optimiser) for qix in range(n_qs)]
    for q in master_qs:
        nocomps = len(q.betas)
        mag = sqrt(sum([ pow(b, 2) for b in q.betas if b!=-10 ]))
        print("Q:{}, difficulty={:.2f} across {} components".format(q.id, mag, nocomps))
    
    for q in master_qs:
        print("qid",q.id,q.betas)
    
    qn_table = BigTable((n_qs, nnw),min_w=beta_min, max_w=beta_max)
    
    return master_qs, qn_table

# Training
This is where sh!t gets real.  We take our tr_len (1000?) students, and iterate over them 100 times to create 100,000 *complete examples* of a student attacking the curriculum.  The questions themselves are attacked in random order: the student has no intelligent guidance throught the material. (Obvious we may wish to provide that guidance at some point in the future.)

Remember, there are only 12 exercises in the curriculum, so if the student is taking 60 or 70 goes to answer them all, that's pretty poor.  But some of these kids are dumb as lumber, so cut them some slack!  They will all get there in the end since by the CMU AFM practice will, eventually, make perfect!

In [269]:
psi_opt = Adam()
def create_students(n_students, nt=n_traits, nnw=n_traits, optimiser=None, seed=666):
    random.seed(seed)
    psi_list = [ Student(psix, theta_min,theta_max, nt=nt, nnw=nnw, optimiser=optimiser) for psix in range(n_students)]
#     for psi in psi_list:
#         print(psi.name, psi.thetas)
        
    psi_table = BigTable((n_students, nnw), min_w=theta_min, max_w=theta_max)
    psi_attn_table = BigTable((n_students, 1), min_w=0, max_w=1)
    print("psi_table wgts", psi_table.get_weights())
    
    return psi_list, psi_table
    

In [270]:
extend_pop=False
extend_by = 90
if extend_pop:
    for _ in range(extend_by):
        nu_psi = Student(nt=n_traits, nq=len(master_qs), optimiser=psi_opt)
        psi_list.append(nu_psi)

In [278]:
import gc
def generate_attempts(master_qs, psi_list, verbose=False):
    attempts =[]
    attempts_by_q = {}
    attempts_by_psi = {}

    user_budget = math.inf
    user_patience = 100
    pass_to_remove = False
    for run in range(1):
        print("----{}\n".format(run))
        for psi in psi_list:
            spend=0
            #psi.mastery = [0 for _ in range(nq)]
            qs = [ix for ix in range(len(master_qs))]
            if verbose:
                print("* * * **** USER {}".format(psi.name))
                print("* * * * ** THETAS {}".format(psi.thetas))

            while(True):
                q_ct = 0
                qix = random.choice(qs)
                q = master_qs[qix]
                passed=False

                if psi.name not in attempts_by_psi:
                    attempts_by_psi[psi.name]=[]

                if q not in attempts_by_q:
                    attempts_by_q[q]=[]

                while not passed and q_ct<user_patience:
                    passed = attempt_q(psi, q, verbose)
                    tup = (psi.id, q.id, passed)
                    attempts.append(tup)
                    attempts_by_psi[psi.name].append(tup)
                    attempts_by_q[q].append(tup)
                    q_ct+=1
                    if (not pass_to_remove):
                        break

                if (not pass_to_remove) or passed:
                    if passed and verbose:
                        print("passed")
                    qs.remove(qix)
                    if verbose:
                        print("removed", qix)

                spend += 1

                if qs == [] or spend>=user_budget:
                        if verbose:
                            print("* ** *QFIN USER {}".format(psi.name))
                        break
    gc.collect()
    return attempts, attempts_by_q, attempts_by_psi

# def generate_q_model(qn_table, optimiser):
#     thetas_in = Input(shape=n_traits, name="thetas_in")
#     qn_sel = Input(shape=(1,), name="q_select", dtype="int32")
#     qn_row = qn_table(qn_sel)
#     dif = subtract([thetas_in, qn_row])
#     Prs = Lambda(lambda z: 1.0 / (1.0 + K.exp(-z)), name="sPr_sigmoid")(dif)
#     Pr = Lambda(lambda ps: K.prod(ps, axis=1, keepdims=True), name="sPr_prod")(Prs)
#     model = Model(inputs=[qn_sel, psi_sel], outputs=Pr)
#     model.compile(optimizer=optimiser, loss="binary_crossentropy", metrics=["mse","accuracy"])
#     return model  

# def generate_psi_model(psi_table, optimiser):
#     betas_in = Input(shape=n_traits, name="thetas_in")
#     psi_sel = Input(shape=(1,), name="psi_select", dtype="int32")
#     psi_row = psi_table(psi_sel)
#     dif = subtract([psi_row, betas_in])
#     Prs = Lambda(lambda z: 1.0 / (1.0 + K.exp(-z)), name="sPr_sigmoid")(dif)
#     Pr = Lambda(lambda ps: K.prod(ps, axis=1, keepdims=True), name="sPr_prod")(Prs)
#     model = Model(inputs=[qn_sel, psi_sel], outputs=Pr)
#     model.compile(optimizer=optimiser, loss="binary_crossentropy", metrics=["mse","accuracy"])
#     return model  


def generate_predictive_model(n_traits, qn_table, psi_table, optimiser):
    psi_in = Input(shape=(n_traits,), name="psi_profile")
    qn_in = Input(shape=(n_traits,), name="q_profile")

#     hidden = concatenate([psi_in,qn_in])
    hidden = subtract([psi_in,qn_in])
#     hidden = concatenate([psi_in,qn_in,hidden])
    hidden = Dense(n_traits, activation="relu")(hidden)
    hidden = Dense(2*n_traits, activation="relu")(hidden)
    hidden = Dense(n_traits, activation="relu")(hidden)
#     hidden = concatenate([hidden,hidden2])
#     hidden = Lambda(lambda z: 1.0 / (1.0 + K.exp(-z)), name="sPr_sigmoid")(hidden)
#     pf = Lambda(lambda ps: K.prod(ps, axis=1, keepdims=True), name="sPr_prod")(hidden)
#     hidden = Dense(10*n_traits, activation="linear")(hidden)

    pf = Dense(1, activation="sigmoid")(hidden)
#     na = Dense(1, activation="relu")(hidden)
    
    model = Model(inputs=[qn_in,psi_in], outputs=pf)
    model.compile(optimizer=optimiser, loss="binary_crossentropy", metrics=["mse","accuracy"])#,"mean_absolute_error"])
    return model

In [279]:
def init_weights(master_qs, psi_list, attempts_by_psi, attempts_by_q, q_table, s_table, max_b, min_th):
    psi_wgts = s_table.get_weights()[0]
    for s in psi_list:
#         attz = [tup[2] for tup in attempts_by_psi[s.name]]
#         prop = mean(attz)
#         p = prop**(1/n_traits)
#         cw_prop = log(p / (1-p))
        psi_wgts[s.id,:] = numpy.random.uniform(min_th,min_th*1.1, size=psi_wgts.shape[1])
        print(psi_wgts[s.id,:])
    s_table.set_weights([ psi_wgts ])

    qn_wgts = q_table.get_weights()[0]
    for q in master_qs:
#         attz = [tup[2] for tup in attempts_by_q[q]]
#         prop = mean(attz)
#         p = prop**(1/n_traits)
#         cw_prop = log((1-p) / p)
        qn_wgts[q.id,:]= numpy.random.uniform(max_b*.9,max_b, size=qn_wgts.shape[1])
        print(qn_wgts[q.id,:])
    q_table.set_weights([ qn_wgts ])
    

In [273]:
import os
import IPython

def calibrate(n_batches, n_students, master_qs, model, n_iter=20, record_param_fit=False):
    es = EarlyStopping(monitor="val_acc", mode="auto", patience=0)
    mc = ModelCheckpoint('best.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

    random.seed(666)
    min_mse = 1000
    min_avg_fit_rmse = math.inf
    min_loss= math.inf
    q_outer_mses = []
    q_outer_accs = []
    s_outer_mses = []
    s_outer_accs = []
    th_mses = []
    b_mses = []
    th_accs= []
    b_accs =[]
    h= []
    avg_fit_rmses = []
    th_fit_rmses = []
    b_fit_rmses = []
    init_patience = 10
    patience = init_patience

#     for pf in pfs:
#         print(pf)
    
#     print(qz)
#     print(sz)
#     for p in pfz:
#         print(p)
    model.summary()
#     input("xuff")
    
    loss = False
    mse = False
    acc = False
    early_stop = False
    min_stop = 1000
    n_iter = 1
    for i in range(n_batches):
        ss, _ = create_students(n_students, n_traits, nnw, optimiser=None, seed=i)
        attempts, attempts_by_q, attempts_by_psi = generate_attempts(qs,ss, verbose=False)

#         print(attempts[0:100])

        q_params = array([qs[tup[1]].betas for tup in attempts]) #reshape(-1,1)
        s_params = array([ss[tup[0]].thetas for tup in attempts]) #reshape(-1,1)
        pfs = array([tup[2] for tup in attempts]).flatten() #reshape(-1,1)
        len_all = q_params.shape[0]

        qz = q_params
        sz = s_params
        pfz = pfs

        base_ix = 0
        done = False
#         for j in range(chunkz+1):
        j = 0
        sub_h = []
          
#         model.train_on_batch(x=[qz, sz], y=pfz)#, epochs=10, shuffle=True, batch_size=1, callbacks=[es])
        model.fit(x=[qz, sz], y=pfz, epochs=100, shuffle=True, batch_size=1, validation_split=0.1, callbacks=[es,mc], verbose=1)

        if i % 10 == 0:
#             print(q_params)
#             print(s_params)
#             print(pfs)
            loss, mse, acc = model.evaluate(x=[q_params, s_params], y=pfz, verbose=0) #, epochs=1, shuffle=True, batch_size=1, verbose=0) #, callbacks=[es])
#             loss2, mse2, acc2 = s_model.evaluate(x=[qices, psices], y=pfs, verbose=0) #, epochs=1, shuffle=True, batch_size=1, verbose=0) #, callbacks=[es])
#             loss=(loss+loss2)/2
#             mse=(mse+mse2)/2
#             acc=(acc+acc2)/2
            sub_h.append((loss,mse,acc))
            print("i=",i)
            print(loss,mse,acc)

    #     del h
    #     loss, mse, acc = qs_model.evaluate(x=[qices, psices], y=pfs)

    #     print(loss, mse, acc)
    return h, th_fit_rmses, b_fit_rmses


In [280]:
def test_predictive_model(m, qs, ss, atts):
    q_params = array([qs[tup[1]].betas for tup in atts]) #reshape(-1,1)
    s_params = array([ss[tup[0]].thetas for tup in atts]) #reshape(-1,1)
    pfs = array([int(tup[2]) for tup in atts]).flatten() #reshape(-1,1)
    len_all = q_params.shape[0]
        
    qz = q_params
    sz = s_params
    pfz = pfs
    
    mr_hats = m.predict(x=[q_params, s_params]) 
    for att,q,s,phat,p in zip(atts,qz,sz,mr_hats,pfz):
        print(ss[att[0]].name, att[1], q,s,p,numpy.round(phat))

    stuff = m.evaluate(x=[qz, sz], y=pfz, verbose=0) #, epochs=1, shuffle=True, batch_size=1, verbose=0) #, callbacks=[es])
    print(stuff)


In [281]:
from keras.models import load_model
nn_dimensions = [n_traits]
serieses = []
min_errs = []
n_qs = 200
n_students = 1000
n_batches = 1
opt = Adam()
for ix,nnw in enumerate(nn_dimensions):
    model = generate_predictive_model(n_traits, q_table, s_table, opt)
    qs, q_table = create_qs(n_qs, n_traits, nnw, optimiser=opt, seed=666)
    h, th_fit_rmses, b_fit_rmses = calibrate(n_batches, n_students,qs, model, n_iter=100, record_param_fit=True)

    model = load_model("best.hdf5")

# for s in ss:
#     print(s.name)

Q:0, difficulty=7.34 across 3 components
Q:1, difficulty=5.25 across 3 components
Q:2, difficulty=3.68 across 3 components
Q:3, difficulty=6.64 across 3 components
Q:4, difficulty=4.77 across 3 components
Q:5, difficulty=2.38 across 3 components
Q:6, difficulty=9.14 across 3 components
Q:7, difficulty=8.92 across 3 components
Q:8, difficulty=6.35 across 3 components
Q:9, difficulty=5.94 across 3 components
Q:10, difficulty=9.52 across 3 components
Q:11, difficulty=6.00 across 3 components
Q:12, difficulty=7.79 across 3 components
Q:13, difficulty=7.86 across 3 components
Q:14, difficulty=3.75 across 3 components
Q:15, difficulty=8.85 across 3 components
Q:16, difficulty=3.26 across 3 components
Q:17, difficulty=7.46 across 3 components
Q:18, difficulty=6.90 across 3 components
Q:19, difficulty=7.70 across 3 components
Q:20, difficulty=7.40 across 3 components
Q:21, difficulty=6.35 across 3 components
Q:22, difficulty=6.90 across 3 components
Q:23, difficulty=5.53 across 3 components
Q:

Train on 180000 samples, validate on 20000 samples
Epoch 1/100
180000/180000 [==============================] - 342s 2ms/step - loss: 0.4835 - mean_squared_error: 0.1584 - acc: 0.7700 - val_loss: 0.4570 - val_mean_squared_error: 0.1479 - val_acc: 0.7890
Epoch 2/100
180000/180000 [==============================] - 308s 2ms/step - loss: 0.4385 - mean_squared_error: 0.1411 - acc: 0.7993 - val_loss: 0.4407 - val_mean_squared_error: 0.1414 - val_acc: 0.7968
Epoch 3/100
180000/180000 [==============================] - 488s 3ms/step - loss: 0.4331 - mean_squared_error: 0.1393 - acc: 0.8011 - val_loss: 0.4392 - val_mean_squared_error: 0.1412 - val_acc: 0.7977
Epoch 4/100
180000/180000 [==============================] - 428s 2ms/step - loss: 0.4326 - mean_squared_error: 0.1392 - acc: 0.8004 - val_loss: 0.4401 - val_mean_squared_error: 0.1416 - val_acc: 0.7980
Epoch 5/100
180000/180000 [==============================] - 583s 3ms/step - loss: 0.4328 - mean_squared_error: 0.1393 - acc: 0.8003 - va

In [282]:
test_qs,_ = create_qs(20, n_traits, nnw, optimiser=opt, seed=42)
test_ss,_ = create_students(200, n_traits, nnw, optimiser=opt, seed=42)
test_attempts, _,_ = generate_attempts(test_qs,test_ss)

test_predictive_model(model, test_qs, test_ss, test_attempts)

Q:0, difficulty=5.45 across 3 components
Q:1, difficulty=5.42 across 3 components
Q:2, difficulty=6.88 across 3 components
Q:3, difficulty=6.69 across 3 components
Q:4, difficulty=4.81 across 3 components
Q:5, difficulty=6.69 across 3 components
Q:6, difficulty=7.68 across 3 components
Q:7, difficulty=2.58 across 3 components
Q:8, difficulty=3.29 across 3 components
Q:9, difficulty=6.87 across 3 components
Q:10, difficulty=3.85 across 3 components
Q:11, difficulty=6.02 across 3 components
Q:12, difficulty=7.32 across 3 components
Q:13, difficulty=8.48 across 3 components
Q:14, difficulty=4.84 across 3 components
Q:15, difficulty=7.42 across 3 components
Q:16, difficulty=2.44 across 3 components
Q:17, difficulty=6.91 across 3 components
Q:18, difficulty=3.68 across 3 components
Q:19, difficulty=6.92 across 3 components
qid 0 [3.1, 2.8, 3.5]
qid 1 [5.4, 0.3, 0.4]
qid 2 [6.4, 2.5, 0.3]
qid 3 [0.0, 3.5, 5.7]
qid 4 [3.5, 1.9, 2.7]
qid 5 [1.2, 4.8, 4.5]
qid 6 [4.8, 5.8, 1.5]
qid 7 [2.4, 0.8,

RECHA JECHYJ  6 [ 4.8  5.8  1.5] [ 4.4  4.   7.8] 0 [ 0.]
RECHA JECHYJ  2 [ 6.4  2.5  0.3] [ 4.4  4.   7.8] 0 [ 0.]
RECHA JECHYJ  4 [ 3.5  1.9  2.7] [ 4.4  4.   7.8] 1 [ 1.]
RECHA JECHYJ  8 [ 2.9  1.2  1. ] [ 4.4  4.   7.8] 1 [ 1.]
RECHA JECHYJ  0 [ 3.1  2.8  3.5] [ 4.4  4.   7.8] 1 [ 1.]
RECHA JECHYJ  10 [ 2.1  0.9  3.1] [ 4.4  4.   7.8] 1 [ 1.]
RECHA JECHYJ  11 [ 3.4  2.8  4.1] [ 4.4  4.   7.8] 1 [ 1.]
RECHA JECHYJ  1 [ 5.4  0.3  0.4] [ 4.4  4.   7.8] 0 [ 0.]
RECHA JECHYJ  3 [ 0.   3.5  5.7] [ 4.4  4.   7.8] 1 [ 1.]
JYH VOSED  19 [ 2.   5.5  3.7] [ 9.1  8.1  9. ] 1 [ 1.]
JYH VOSED  7 [ 2.4  0.8  0.5] [ 9.1  8.1  9. ] 1 [ 1.]
JYH VOSED  13 [ 2.7  5.   6.3] [ 9.1  8.1  9. ] 1 [ 1.]
JYH VOSED  1 [ 5.4  0.3  0.4] [ 9.1  8.1  9. ] 1 [ 1.]
JYH VOSED  6 [ 4.8  5.8  1.5] [ 9.1  8.1  9. ] 1 [ 1.]
JYH VOSED  10 [ 2.1  0.9  3.1] [ 9.1  8.1  9. ] 1 [ 1.]
JYH VOSED  18 [ 3.4  1.4  0.1] [ 9.1  8.1  9. ] 1 [ 1.]
JYH VOSED  17 [ 4.9  4.8  0.8] [ 9.1  8.1  9. ] 1 [ 1.]
JYH VOSED  4 [ 3.5  1.9  2.7] [

KOCYK MYG  7 [ 2.4  0.8  0.5] [ 9.2  9.   3.7] 1 [ 1.]
FAV PHANIJ  18 [ 3.4  1.4  0.1] [ 4.9  8.1  8.8] 0 [ 1.]
FAV PHANIJ  17 [ 4.9  4.8  0.8] [ 4.9  8.1  8.8] 1 [ 0.]
FAV PHANIJ  19 [ 2.   5.5  3.7] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  2 [ 6.4  2.5  0.3] [ 4.9  8.1  8.8] 0 [ 0.]
FAV PHANIJ  7 [ 2.4  0.8  0.5] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  9 [ 4.7  4.6  2. ] [ 4.9  8.1  8.8] 0 [ 0.]
FAV PHANIJ  6 [ 4.8  5.8  1.5] [ 4.9  8.1  8.8] 1 [ 0.]
FAV PHANIJ  14 [ 3.1  1.7  3.3] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  5 [ 1.2  4.8  4.5] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  4 [ 3.5  1.9  2.7] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  13 [ 2.7  5.   6.3] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  10 [ 2.1  0.9  3.1] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  3 [ 0.   3.5  5.7] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  8 [ 2.9  1.2  1. ] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  12 [ 4.   3.4  5.1] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  11 [ 3.4  2.8  4.1] [ 4.9  8.1  8.8] 1 [ 1.]
FAV PHANIJ  1 [ 5.4  0.3  0.4] [ 4.9  8.1

LOTH SISOP  15 [ 2.8  4.6  5.1] [ 5.7  9.5  5.2] 1 [ 1.]
LOTH SISOP  18 [ 3.4  1.4  0.1] [ 5.7  9.5  5.2] 1 [ 1.]
LOTH SISOP  10 [ 2.1  0.9  3.1] [ 5.7  9.5  5.2] 1 [ 1.]
LOTH SISOP  4 [ 3.5  1.9  2.7] [ 5.7  9.5  5.2] 1 [ 1.]
LOTH SISOP  12 [ 4.   3.4  5.1] [ 5.7  9.5  5.2] 1 [ 0.]
LOTH SISOP  1 [ 5.4  0.3  0.4] [ 5.7  9.5  5.2] 0 [ 1.]
LOTH SISOP  2 [ 6.4  2.5  0.3] [ 5.7  9.5  5.2] 1 [ 0.]
LOTH SISOP  7 [ 2.4  0.8  0.5] [ 5.7  9.5  5.2] 1 [ 1.]
LOTH SISOP  6 [ 4.8  5.8  1.5] [ 5.7  9.5  5.2] 1 [ 1.]
LOTH SISOP  17 [ 4.9  4.8  0.8] [ 5.7  9.5  5.2] 1 [ 1.]
BUBYF COHI  11 [ 3.4  2.8  4.1] [ 7.9  8.4  7.1] 1 [ 1.]
BUBYF COHI  8 [ 2.9  1.2  1. ] [ 7.9  8.4  7.1] 1 [ 1.]
BUBYF COHI  15 [ 2.8  4.6  5.1] [ 7.9  8.4  7.1] 1 [ 1.]
BUBYF COHI  6 [ 4.8  5.8  1.5] [ 7.9  8.4  7.1] 1 [ 1.]
BUBYF COHI  5 [ 1.2  4.8  4.5] [ 7.9  8.4  7.1] 0 [ 1.]
BUBYF COHI  4 [ 3.5  1.9  2.7] [ 7.9  8.4  7.1] 1 [ 1.]
BUBYF COHI  16 [ 0.6  1.4  1.9] [ 7.9  8.4  7.1] 1 [ 1.]
BUBYF COHI  19 [ 2.   5.5  3.7] [ 7.9  8

DOW TOLOV  0 [ 3.1  2.8  3.5] [ 4.3  4.8  6.2] 0 [ 1.]
DOW TOLOV  6 [ 4.8  5.8  1.5] [ 4.3  4.8  6.2] 0 [ 0.]
DOW TOLOV  16 [ 0.6  1.4  1.9] [ 4.3  4.8  6.2] 1 [ 1.]
DOW TOLOV  17 [ 4.9  4.8  0.8] [ 4.3  4.8  6.2] 1 [ 0.]
DOW TOLOV  9 [ 4.7  4.6  2. ] [ 4.3  4.8  6.2] 1 [ 0.]
DOW TOLOV  7 [ 2.4  0.8  0.5] [ 4.3  4.8  6.2] 1 [ 1.]
DOW TOLOV  15 [ 2.8  4.6  5.1] [ 4.3  4.8  6.2] 0 [ 0.]
DOW TOLOV  2 [ 6.4  2.5  0.3] [ 4.3  4.8  6.2] 0 [ 0.]
DOW TOLOV  5 [ 1.2  4.8  4.5] [ 4.3  4.8  6.2] 0 [ 0.]
DOW TOLOV  4 [ 3.5  1.9  2.7] [ 4.3  4.8  6.2] 0 [ 1.]
DOW TOLOV  11 [ 3.4  2.8  4.1] [ 4.3  4.8  6.2] 1 [ 1.]
DOW TOLOV  3 [ 0.   3.5  5.7] [ 4.3  4.8  6.2] 1 [ 0.]
DOW TOLOV  19 [ 2.   5.5  3.7] [ 4.3  4.8  6.2] 1 [ 0.]
DOW TOLOV  8 [ 2.9  1.2  1. ] [ 4.3  4.8  6.2] 1 [ 1.]
DOW TOLOV  10 [ 2.1  0.9  3.1] [ 4.3  4.8  6.2] 1 [ 1.]
DOW TOLOV  18 [ 3.4  1.4  0.1] [ 4.3  4.8  6.2] 0 [ 1.]
DOW TOLOV  14 [ 3.1  1.7  3.3] [ 4.3  4.8  6.2] 1 [ 1.]
DOW TOLOV  1 [ 5.4  0.3  0.4] [ 4.3  4.8  6.2] 0 [ 0.]
DO

...

NOGYW FALEF  0 [ 3.1  2.8  3.5] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  6 [ 4.8  5.8  1.5] [ 10.1   8.5   8.9] 0 [ 1.]
NOGYW FALEF  9 [ 4.7  4.6  2. ] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  1 [ 5.4  0.3  0.4] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  19 [ 2.   5.5  3.7] [ 10.1   8.5   8.9] 0 [ 1.]
NOGYW FALEF  14 [ 3.1  1.7  3.3] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  13 [ 2.7  5.   6.3] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  12 [ 4.   3.4  5.1] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  10 [ 2.1  0.9  3.1] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  2 [ 6.4  2.5  0.3] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  3 [ 0.   3.5  5.7] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  15 [ 2.8  4.6  5.1] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  18 [ 3.4  1.4  0.1] [ 10.1   8.5   8.9] 1 [ 1.]
NOGYW FALEF  16 [ 0.6  1.4  1.9] [ 10.1   8.5   8.9] 1 [ 1.]
VOSUY GIG  18 [ 3.4  1.4  0.1] [ 7.6  9.3  4.2] 1 [ 1.]
VOSUY GIG  12 [ 4.   3.4  5.1] [ 7.6  9.3  4.2] 0 [ 0.]
VOSUY GIG  8 [ 2.9  1.2  1. ] [ 7.6  9.3

HIL VUMA  2 [ 6.4  2.5  0.3] [ 4.1  4.8  9.4] 0 [ 0.]
HIL VUMA  19 [ 2.   5.5  3.7] [ 4.1  4.8  9.4] 1 [ 0.]
HIL VUMA  9 [ 4.7  4.6  2. ] [ 4.1  4.8  9.4] 1 [ 0.]
HIL VUMA  8 [ 2.9  1.2  1. ] [ 4.1  4.8  9.4] 1 [ 1.]
HIL VUMA  11 [ 3.4  2.8  4.1] [ 4.1  4.8  9.4] 0 [ 1.]
HIL VUMA  13 [ 2.7  5.   6.3] [ 4.1  4.8  9.4] 0 [ 0.]
HIL VUMA  12 [ 4.   3.4  5.1] [ 4.1  4.8  9.4] 0 [ 0.]
HIL VUMA  14 [ 3.1  1.7  3.3] [ 4.1  4.8  9.4] 0 [ 1.]
HIL VUMA  4 [ 3.5  1.9  2.7] [ 4.1  4.8  9.4] 1 [ 1.]
HIL VUMA  5 [ 1.2  4.8  4.5] [ 4.1  4.8  9.4] 0 [ 0.]
HIL VUMA  18 [ 3.4  1.4  0.1] [ 4.1  4.8  9.4] 1 [ 1.]
HIL VUMA  16 [ 0.6  1.4  1.9] [ 4.1  4.8  9.4] 1 [ 1.]
HIL VUMA  15 [ 2.8  4.6  5.1] [ 4.1  4.8  9.4] 0 [ 0.]
HIL VUMA  0 [ 3.1  2.8  3.5] [ 4.1  4.8  9.4] 1 [ 1.]
HIL VUMA  7 [ 2.4  0.8  0.5] [ 4.1  4.8  9.4] 1 [ 1.]
HIL VUMA  10 [ 2.1  0.9  3.1] [ 4.1  4.8  9.4] 0 [ 1.]
HIL VUMA  17 [ 4.9  4.8  0.8] [ 4.1  4.8  9.4] 0 [ 0.]
GOPH FIK  7 [ 2.4  0.8  0.5] [ 4.7  8.2  6.8] 1 [ 1.]
GOPH FIK  4 [ 3.5 

SEGA DYBAC  3 [ 0.   3.5  5.7] [ 9.6  8.2  8.5] 1 [ 1.]
SEGA DYBAC  13 [ 2.7  5.   6.3] [ 9.6  8.2  8.5] 1 [ 1.]
SEGA DYBAC  4 [ 3.5  1.9  2.7] [ 9.6  8.2  8.5] 1 [ 1.]
SEGA DYBAC  15 [ 2.8  4.6  5.1] [ 9.6  8.2  8.5] 1 [ 1.]
SEGA DYBAC  12 [ 4.   3.4  5.1] [ 9.6  8.2  8.5] 1 [ 1.]
CYY MOMA  9 [ 4.7  4.6  2. ] [  8.3   5.1  10. ] 1 [ 1.]
CYY MOMA  13 [ 2.7  5.   6.3] [  8.3   5.1  10. ] 1 [ 1.]
CYY MOMA  19 [ 2.   5.5  3.7] [  8.3   5.1  10. ] 0 [ 0.]
CYY MOMA  1 [ 5.4  0.3  0.4] [  8.3   5.1  10. ] 1 [ 1.]
CYY MOMA  0 [ 3.1  2.8  3.5] [  8.3   5.1  10. ] 1 [ 1.]
CYY MOMA  3 [ 0.   3.5  5.7] [  8.3   5.1  10. ] 0 [ 1.]
CYY MOMA  10 [ 2.1  0.9  3.1] [  8.3   5.1  10. ] 1 [ 1.]
CYY MOMA  7 [ 2.4  0.8  0.5] [  8.3   5.1  10. ] 1 [ 1.]
CYY MOMA  11 [ 3.4  2.8  4.1] [  8.3   5.1  10. ] 1 [ 1.]
CYY MOMA  16 [ 0.6  1.4  1.9] [  8.3   5.1  10. ] 1 [ 1.]
CYY MOMA  15 [ 2.8  4.6  5.1] [  8.3   5.1  10. ] 1 [ 1.]
CYY MOMA  14 [ 3.1  1.7  3.3] [  8.3   5.1  10. ] 1 [ 1.]
CYY MOMA  18 [ 3.4  1.4  0

VEJETH JOGEM  8 [ 2.9  1.2  1. ] [ 8.1  3.9  8.1] 1 [ 1.]
VEJETH JOGEM  12 [ 4.   3.4  5.1] [ 8.1  3.9  8.1] 0 [ 1.]
VEJETH JOGEM  17 [ 4.9  4.8  0.8] [ 8.1  3.9  8.1] 1 [ 0.]
VEJETH JOGEM  0 [ 3.1  2.8  3.5] [ 8.1  3.9  8.1] 0 [ 1.]
VEJETH JOGEM  5 [ 1.2  4.8  4.5] [ 8.1  3.9  8.1] 1 [ 0.]
VEJETH JOGEM  1 [ 5.4  0.3  0.4] [ 8.1  3.9  8.1] 1 [ 1.]
VEJETH JOGEM  18 [ 3.4  1.4  0.1] [ 8.1  3.9  8.1] 1 [ 1.]
VEJETH JOGEM  6 [ 4.8  5.8  1.5] [ 8.1  3.9  8.1] 0 [ 0.]
VEJETH JOGEM  7 [ 2.4  0.8  0.5] [ 8.1  3.9  8.1] 1 [ 1.]
VEJETH JOGEM  13 [ 2.7  5.   6.3] [ 8.1  3.9  8.1] 0 [ 0.]
FETHEC JET  12 [ 4.   3.4  5.1] [ 9.7  3.6  7.8] 0 [ 1.]
FETHEC JET  14 [ 3.1  1.7  3.3] [ 9.7  3.6  7.8] 0 [ 1.]
FETHEC JET  1 [ 5.4  0.3  0.4] [ 9.7  3.6  7.8] 1 [ 1.]
FETHEC JET  4 [ 3.5  1.9  2.7] [ 9.7  3.6  7.8] 1 [ 1.]
FETHEC JET  3 [ 0.   3.5  5.7] [ 9.7  3.6  7.8] 1 [ 1.]
FETHEC JET  8 [ 2.9  1.2  1. ] [ 9.7  3.6  7.8] 1 [ 1.]
FETHEC JET  19 [ 2.   5.5  3.7] [ 9.7  3.6  7.8] 1 [ 0.]
FETHEC JET  5 [ 1.2  

[0.42810715904831886, 0.13780130059365184, 0.80400000000000005]


In [277]:
print("elements in h:", len(h))
for tup in h:
    print(tup)

loss, mse, acc = zip(*h)


fig = plt.gcf()
#     plt.xlabel("#iterations")
#     plt.ylabel("fit error (RMSE)")
#     plt.suptitle("Neural-MLTM Parameter Fitting")
#     plt.title("(skills=5, items=10, students=100)")
fig, axes = plt.subplots(nrows=1, ncols=3)
axes[0].plot(acc, label="acc")
axes[1].plot(loss, label="loss")
axes[2].plot(mse, label="mse")

fig.set_size_inches(20, 5)
for i in [0,1,2]:
    axes[i].legend()
plt.show()

av_fit_rmses = []
for b,th in zip(b_fit_rmses, th_fit_rmses):
    av = (b+th)/2.0
    av_fit_rmses.append(av)
    
plt.plot(b_fit_rmses, label="beta fit")
plt.plot(th_fit_rmses, label="theta fit")
plt.plot(av_fit_rmses, label="av")
plt.legend()
plt.show()
# fig = plt.gcf()
# fig.set_size_inches(8, 5)
# plt.xlabel("#iterations")
# plt.ylabel("fit error (RMSE)")
# plt.suptitle("Neural-MLTM Parameter Fitting")
# plt.title("(skills=5, items=10, students=100)")
# plt.legend()
# plt.show()


elements in h: 0


ValueError: not enough values to unpack (expected 3, got 0)

In [ ]:
from scipy.spatial.distance import cosine

real_wgts = array([ q.betas for q in qs ])
pred_wgts = q_table.get_weights()[0]
# pred_wgts = numpy.round(pred_wgts,1)

out_cols = [None] * len(real_wgts.T)
curr_sel = None
curr_ix = None
n_iters = 10
chosen = None

indices = range(len(real_wgts.T))

min_total_err = math.inf
for i in range(100): #len(indices)**2):
    print("i is ",i)
    real_used = set()
    pred_used = set()
    while len(pred_used) < len(indices):
        curr_mse = math.inf
        for rix in numpy.random.permutation(indices):
            if rix in real_used:
                continue
            real_col = real_wgts.T[rix]
            for cix in numpy.random.permutation(indices):
                if cix in pred_used:
                    continue
                pred_col = pred_wgts.T[cix]
                mse = numpy.mean(numpy.abs( pred_col - real_col))
#                 mse = cosine(pred_col, real_col)
                print("mae is ",mse)
                if mse < curr_mse:
                    print("best match", cix, rix)
                    print(real_col)
                    print(pred_col)
                    curr_sel = pred_col
                    curr_mse = mse
                    curr_ix = cix
                    curr_real_ix = rix
        print("---")
        real_used.add(curr_real_ix)
        pred_used.add(curr_ix)
        out_cols[curr_real_ix] = curr_sel
    out_col_arr = array(out_cols).T
    total_err = numpy.mean(numpy.abs( out_col_arr - real_wgts ))
#     total_err = cosine(out_col_arr.flatten(), real_wgts.flatten())
    mean_ll = numpy.mean( out_col_arr - real_wgts )
    if total_err < min_total_err:
        min_total_err = total_err
        best_ll = mean_ll
        chosen = out_col_arr
        print("new total min mae:", min_total_err)
        print("new best ll", best_ll)
        
print("real", real_wgts)
# print(pred_wgts)
print("out", chosen)
print("elementwise mae:", min_total_err)
print("mean lead/lag", mean_ll)


In [ ]:
fig = plt.gcf()
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

# pca = PCA(n_components=2)
# pca2 = PCA(n_components=2)
pca = TSNE(n_components=2)
pca2 = TSNE(n_components=2)

itemz = array([ q.betas for q in qs ])

itemz_2 = itemz
itemz_2 = pca.fit_transform(itemz_2)
itemz_2 = MinMaxScaler().fit_transform(itemz_2)

itemz_pred = chosen
itemz_pred = pca.fit_transform(itemz_pred)
itemz_pred = MinMaxScaler().fit_transform(itemz_pred)
# print(itemz_2)

# fig,axs = plt.subplots(1,2)
fig = plt.gcf()
fig.set_size_inches(15, 10)

for x,xh,y,yh in zip(itemz_2[:,0],itemz_pred[:,0],itemz_2[:,1],itemz_pred[:,1]):
    fig.gca().plot([x,xh],[y,yh],color="#aaaaaa")

fig.gca().scatter(itemz_2[:,0], itemz_2[:,1], alpha=0.7)
fig.gca().scatter(itemz_pred[:,0], itemz_pred[:,1], alpha=0.5)

for i, txt in enumerate(itemz_2):
    fig.gca().annotate(i, (itemz_2[i,0], itemz_2[i,1]))

for i, txt in enumerate(itemz_pred):
    fig.gca().annotate(i, (itemz_pred[i,0], itemz_pred[i,1]))
    
fig.show()

In [ ]:
psi_wgts = s_table.get_weights()[0]
print(psi_wgts)
for psi in ss:
    print(psi.id, psi.thetas, psi_wgts[psi.id])
print("qs ====")
qn_wgts = q_table.get_weights()[0]
print(qn_wgts)
for qn in qs:
    print(qn.id, qn.betas, qn_wgts[qn.id])

In [ ]:
n_traits = 5
nn_dimensions = [1,3,5,7,9]
serieses = []
min_errs = []
n_qs = 10
n_students = 250
for ix,nnw in enumerate(nn_dimensions):
    qs = create_qs(n_qs, n_traits, nnw, optimiser=qopt)
    ss = create_students(n_students, n_traits, nnw, optimiser=qopt)
    attempts_by_psi, attempts_by_q = generate_attempts(qs,ss)
    print(qs[0].pred_betas.get_weights()[0][0])
    print(ss[0].pred_theta.get_weights()[0][0])
    resultz = calibrate(qs,ss,attempts_by_q, attempts_by_psi, n_iter=70)
    serieses.append( resultz )
#     if ix < len(serieses): #append to old series
#         (_mses,_accs) = serieses[ix]
#         _mses += mses
#         _accs += accs
#         serieuses[ix] = (_mses,_accs)
#     else: #create new series
#         serieses.append((mses,accs))
    
from pygame import mixer
mixer.init()
mixer.music.load('calibration_complete.mp3')
mixer.music.play()

In [ ]:
# import pickle
# pickle.dump( serieses, open( "serieses.p", "wb" ) )

xmax=10
xs = range(len(serieses[0][0]))[0:xmax]
print(len(serieses))
for s in serieses:
    print(len(s))
# plt.plot(xs, numpy.multiply(1,theta_rmses), 'b--')
# plt.plot(xs, numpy.multiply(1,beta_rmses), 'b')
fig, axes = plt.subplots(nrows=3, ncols=2)
print(axes.shape)
for (mses,accs, th_mses,th_accs, b_mses,b_accs),c,d in zip(serieses,["r--","y--","k-","c--","b--"],nn_dimensions):
    axes[0,0].plot(xs, numpy.multiply(1,mses[0:xmax]), c, label="nnw={}".format(d))
    axes[0,1].plot(xs, numpy.multiply(1,accs[0:xmax]), c, label="nnw={}".format(d))
    axes[1,0].plot(xs, numpy.multiply(1,th_mses[0:xmax]), c, label="nnw={}".format(d))
    axes[1,1].plot(xs, numpy.multiply(1,th_accs[0:xmax]), c, label="nnw={}".format(d))
    axes[2,0].plot(xs, numpy.multiply(1,b_mses[0:xmax]), c, label="nnw={}".format(d))
    axes[2,1].plot(xs, numpy.multiply(1,b_accs[0:xmax]), c, label="nnw={}".format(d))

# plt.plot(xs, accs, "m")
# plt.plot(xs, numpy.multiply(1,s_outer_accs), "g")7
# plt.plot(xs, numpy.multiply(1,q_outer_accs), "k")

# plt.plot(xs, numpy.multiply(1,mses), "m--")
# plt.plot(xs, numpy.multiply(1,s_outer_mses), "g--")
# plt.plot(xs, numpy.multiply(1,q_outer_mses), "k--")
for ix in range(axes.shape[0]):
    subcats = ["Combined","Student","Question"]
    for iy in range(axes.shape[1]):
        axes[ix,iy].set_xlabel("#iterations")
        axes[ix,iy].legend()
        if iy==0:
            axes[ix,iy].set_title("{} fit error".format(subcats[ix]))
            axes[ix,iy].set_ylabel("fit error (RMSE)")
        else:
            axes[ix,iy].set_title("{} fit accuracy".format(subcats[ix]))
            axes[ix,iy].set_ylabel("prediction accuracy")

fig.suptitle("Neural-MLTM Parameter Fitting (k={}, q={}, s={})".format(n_traits, n_qs, n_students))
fig.set_size_inches(18, 18)

fig.show()

In [ ]:
xs = range(len(serieses[0][0]))
print(len(serieses))
for s in serieses:
    print(len(s))
# plt.plot(xs, numpy.multiply(1,theta_rmses), 'b--')
# plt.plot(xs, numpy.multiply(1,beta_rmses), 'b')
fig, axes = plt.subplots(nrows=1, ncols=2)
print(axes.shape)

min_mses = []
max_accs = []
bmin_mses = []
bmax_accs = []

for (mses,accs, th_mses,th_accs, b_mses,b_accs),c,d in zip(serieses,["r--","y--","k-","c--","b--"],nn_dimensions):
    mm = min(th_mses)
    ma = max(th_accs)
    min_mses.append(mm)
    max_accs.append(ma)
    mm = min(b_mses)
    ma = max(b_accs)
    bmin_mses.append(mm)
    bmax_accs.append(ma)
    

axes[0].plot(nn_dimensions, numpy.multiply(1,min_mses))
axes[0].plot(nn_dimensions, numpy.multiply(1,bmin_mses))
axes[1].plot(nn_dimensions, numpy.multiply(1,max_accs))
axes[1].plot(nn_dimensions, numpy.multiply(1,bmax_accs))
axes[0].axvline(x=5, linestyle="--")
axes[1].axvline(x=5, linestyle="--")

fig.suptitle("Neural-MLTM Parameter Fitting (k={}, q={}, s={})".format(n_traits, n_qs, n_students))
fig.set_size_inches(18,4)

fig.show()